In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import re
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
import seaborn as sns
import copy
from sklearn import preprocessing
import pickle
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, HessianInversionWarning

In [2]:
# Filtering out warnings in case they appear to prevent flooding of outputs
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', HessianInversionWarning)
warnings.simplefilter('ignore', pd.errors.DtypeWarning)
warnings.simplefilter('ignore', RuntimeWarning)


In [17]:
def compute_ci_95(arr):
    """
    Helper function to calculate 95% interval given an array
    """
    return [np.percentile(arr, 2.5), np.percentile(arr, 97.5)]

### Merge full mimiic data with mimic data predictions for smoking status

In [3]:
# Retrieving merged_data -- can't show due to MIMIC Privacy Policy
full_data_df = pd.read_csv("full_data_df_no_index.csv")
pred_mimic_df = pd.read_csv("...")# Should be the csv file with mimic smoking status predictions for each entry
pred_mimic_df = pred_mimic_df.rename(columns={'SUBJECT_ID': 'subject_id'})

#### full_data_df contains 6361 rows and 130 columns (including subject_id, age, echo, etc...)
#### pred_mimic_df contains 34312 rows and 46 columns (including subject_id, SMOKING_STATUS, etc..)

In [4]:
merged_df = pd.merge(full_data_df, pred_mimic_df[["subject_id","SMOKING_STATUS"]], on=["subject_id"])

In [5]:
merged_df["SMOKING_STATUS"].value_counts()

1    2058
3    1413
4    1171
2      93
0      64
Name: SMOKING_STATUS, dtype: int64

In [6]:
# Droppping 0 labels to ensure we only have 4 possible smoking status labels
# 0 labels derived from merging two dataframes where some entries may not have a prediction
merged_df = merged_df.drop(merged_df[merged_df["SMOKING_STATUS"] == 0].index)
merged_df["SMOKING_STATUS"].value_counts()

1    2058
3    1413
4    1171
2      93
Name: SMOKING_STATUS, dtype: int64

In [7]:
# Converting integers to weekdays
def int_to_weekday(row):
    r = int(row)
    if r == 0:
        return 'sunday'
    elif r == 1:
        return "monday"
    elif r == 2:
        return "tuesday"
    elif r == 3:
        return "wednesday"
    elif r == 4:
        return "thursday"
    elif r== 5:
        return "friday"
    else:
        return "saturday"

merged_df["icu_adm_weekday"] = merged_df["icu_adm_weekday"].apply(int_to_weekday)

In [8]:
merged_df["first_careunit"] = merged_df["first_careunit"].astype('category')
merged_df["first_careunit"] = merged_df["first_careunit"].cat.reorder_categories(["SICU", "MICU"])

merged_df["gender"] = merged_df["gender"].astype("category")
merged_df["gender"] = merged_df["gender"].cat.reorder_categories(["M", "F"])

merged_df["icu_adm_weekday"] = merged_df["icu_adm_weekday"].astype("category")

### Viewing the finalized merged dataframe

#### 4735 rows and 131 columns (including subject_id, echo, mort_28_day, SMOKING_STATUS, etc...)

In [ ]:
merged_df # Unable to show due to MIMIC Privacy Policy

### Defining helper functions to calculate causal effects w.r.t effect restoration from measurement bias

In [9]:
def generate_models(dataframe):
    '''
    Given a pre-processed MIMIC + proxy prediction dataframe, train three logistic regression models 
    using smf.logit. 
    The formula strings will be hard-coded into the function. The assumptions for these models are:
        1) Categorical smoking categories 
        2) Not all feature are binary, but at least the output (mort_28_day) and treatment (echo) 
           should be binary
    '''
    
    # Calculating P(y | u*, a, c) --> y ~ u* + a + c for each u* label in [1,2,3,4] 
    fstring = 'mort_28_day ~ echo + age + weight + saps + sofa + elix_score + vent + \
            vaso + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \
            icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \
            lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_chloride_first + \
            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_creatinine_first + \
            lab_potassium_first + vs_cvp_flag + lab_creatinine_kinase_flag + lab_bnp_flag + gender + \
            lab_troponin_flag + first_careunit + icu_adm_weekday + lab_ph_first + lab_pco2_first + \
            lab_po2_first + lab_lactate_first + sedative + C(SMOKING_STATUS)'
    eq1 = smf.logit(fstring, data=dataframe)
    eq1_model = eq1.fit(disp=0)
    
    # Calculating P(u* | a, c)
    f_string2 = "SMOKING_STATUS ~ echo + first_careunit + age + gender + weight + saps + sofa + elix_score + \
            vent + vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + \
            icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + \
            vs_temp_first + lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + \
            lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \
            lab_chloride_first + lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + \
            vs_cvp_flag + lab_creatinine_kinase_flag + lab_bnp_flag + lab_troponin_flag"
    eq2 = smf.mnlogit(f_string2, data=dataframe)
    eq2_model = eq2.fit(disp=0)
    
    # Calculating P(a|c)
    f_string3 = "echo ~ first_careunit + age + gender + weight + saps + sofa + elix_score + vent + \
                vaso + icu_adm_weekday + icu_adm_hour + icd_chf + icd_afib + icd_renal + icd_liver + icd_copd + \
                icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_map_first + vs_temp_first + \
                lab_hemoglobin_first + lab_platelet_first + lab_wbc_first + lab_ph_first + lab_chloride_first + \
                lab_sodium_first + lab_bun_first + lab_bicarbonate_first + lab_pco2_first + lab_creatinine_first + \
                lab_potassium_first + lab_po2_first + lab_lactate_first + sedative + vs_cvp_flag + \
                lab_creatinine_kinase_flag + lab_bnp_flag + lab_troponin_flag"
    eq3 = smf.logit(f_string3, data=dataframe)
    eq3_model = eq3.fit(disp=0)
    
    return eq1_model, eq2_model, eq3_model

### Implementing Risk Ratio

In [30]:
def risk_ratio(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), calculate the risk ratio as defined by: 
    causal_effect = summation(c,u){ p(c,u) * ( E[Y=1 | A=1,c,u*] / E[Y=1 | A=0,c,u*] ) }
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 for not 
           receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
        4) Default prediction is probability of getting 1 due to how statsmodels works
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    # Understanding Matrix of Error Adjustments
    confusion = [
                    [8, 0, 2, 1],
                    [4, 4, 3, 0],
                    [1, 0, 14, 1],
                    [1, 0, 1, 61]
                ] # rows represent the ground truth labels and cols represents the predicted labels

    error_mat = [
                    [8/11, 0, 2/11, 1/11],
                    [4/11, 4/11, 3/11, 0],
                    [1/16, 0, 14/16, 1/16],
                    [1/63, 0, 1/63, 61/63]
                ] # rows represent U* and cols represent U
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
        
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    comp_0 = upper_0a / lower_0b
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    comp_1 = upper_1a / lower_1b
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    comp_2 = upper_2a / lower_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    comp_3 = upper_3a / lower_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    rr = np.mean(comp_0 * prob_u0_c) + np.mean(comp_1 * prob_u1_c) + np.mean(comp_2 * prob_u2_c) \
            + np.mean(comp_3 * prob_u3_c)
    sub_array = [np.mean(comp_0), np.mean(comp_1), np.mean(comp_2), np.mean(comp_3)]
    return rr, sub_array

In [31]:
m1, m2, m3 = generate_models(merged_df)
risk_ratio(merged_df, m1, m2, m3)

(0.8768839707583874,
 [0.8615914941715498,
  0.9221790334361804,
  1.1265191373553538,
  0.880485625812281])

### Bootstrapping Merged Dataframe

In [32]:
def bootstrap_merged_data_rr(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), perform bootstrapping by shuffling the merged dataframe for
    risk ratio calculations. Iterations set to 100.
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    
    
    iterations = 100
    output = []
    sub_matrix = np.zeros((iterations, 4))
    for i in range(iterations):
        bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
        rr, sub_array = risk_ratio(bt_df, model1, model2, model3)
        output.append(rr)

        for idx, c in enumerate(sub_array):
            sub_matrix[i, idx] = c
        sub_avg = sub_matrix.mean(axis=0)
        
    res_dict = {"bs_rr": sum(output) / len(output), "bs_arr_rr": output, "sub_avg_rr": sub_avg, "sub_arr_rr": sub_matrix}
    
    return res_dict

bt_merged_rr = bootstrap_merged_data_rr(merged_df, m1, m2, m3)
bt_merged_rr

{'bs_rr': 0.8752277806789786,
 'bs_arr_rr': [0.8792620765965039,
  0.879740100838371,
  0.8782733297481601,
  0.8428956143727323,
  0.8733607375901483,
  0.871982494989969,
  0.8722203887567788,
  0.8888133088559683,
  0.8903239853159824,
  0.8611063945630258,
  0.8773958964255891,
  0.8885152502074745,
  0.8534523209111112,
  0.8654019470719876,
  0.8723176785933369,
  0.86504731169704,
  0.8871564446671825,
  0.8839019256220305,
  0.8699253451202827,
  0.8772146748420085,
  0.8645474507844941,
  0.885143805924025,
  0.8697895485713791,
  0.8828761369951353,
  0.8730714832253195,
  0.8804504075019648,
  0.8715413568544357,
  0.8583918139276203,
  0.8714038787433396,
  0.8920070306932663,
  0.8718193576318638,
  0.8859767367134652,
  0.88121141777159,
  0.8805594373438067,
  0.8684900444718341,
  0.8608751470984348,
  0.8823092030923647,
  0.8801510079093401,
  0.873426441179112,
  0.8803368931089892,
  0.8850021696699721,
  0.871188462493574,
  0.8766819885398288,
  0.8464785991440389

In [33]:
# Computing 95% interval for risk ratio while boostrapping merged dataframe 
compute_ci_95(bt_merged_rr["bs_arr_rr"])

[0.8498027651255051, 0.8925575893480476]

In [34]:
# Computing 95% interval for risk ratio subgroups while bootstrapping merged dataframe
np.apply_along_axis(compute_ci_95, axis=0, arr=bt_merged_rr["sub_arr_rr"])

array([[0.76900858, 0.92050188, 0.65553224, 0.63700515],
       [0.92529492, 0.92389376, 1.84907032, 0.98958382]])

### Bootstrapping Error Rate Matrix

In [35]:
def risk_ratio_bootstrap(dataframe, model1, model2, model3, error_mat):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe, three trained models 
    from generate_models(), and an error rate matrix, perform bootstrapping on error-rate matrix for
    risk-ratio calculations. Helper function for bootstrap()
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    # Inversing Error Rate Matrices
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
        
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    comp_0 = upper_0a / lower_0b
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    comp_1 = upper_1a / lower_1b
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    comp_2 = upper_2a / lower_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    comp_3 = upper_3a / lower_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    rr = np.mean(comp_0 * prob_u0_c) + np.mean(comp_1 * prob_u1_c) + np.mean(comp_2 * prob_u2_c) \
        + np.mean(comp_3 * prob_u3_c)
    sub_array = [np.mean(comp_0), np.mean(comp_1), np.mean(comp_2), np.mean(comp_3)]
    return rr, sub_array

In [36]:
def bootstrap(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), bootstrap the testing set 
    for n2c2 2006 smoking dataset to get different error rate matrices to test robustness of the 
    risk ratio casual effect. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterating through the bootstrapped confusion matrices 
    # "iterations" var depends on how many bootstrapped confusion matrics were generated
    # Default in predict_bootstrap_2006.py is 10
    iterations = 100 
    rr_arr = []
    sub_matrix = np.zeros((iterations, 4))

    for x in range(iterations):
        # Access each pickle file containing the confusion matrix
        f = open("...", "rb")  # First input should be the bootstrapped matrices (pkl file)
        con_matrix = pickle.load(f)
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        rr, sub_array = risk_ratio_bootstrap(dataframe, model1, model2, model3, res)
        rr_arr.append(rr)
        
        for idx, c in enumerate(sub_array):
            sub_matrix[x, idx] = c
        sub_avg = sub_matrix.mean(axis=0)
        
    res_dict = {"bt_rr": sum(rr_arr) / len(rr_arr), "bt_arr_rr": rr_arr, "sub_avg_rr": sub_avg, "sub_arr_rr": sub_matrix}
    
    return res_dict

In [39]:
m1, m2, m3 = generate_models(merged_df)
bt_rr = bootstrap(merged_df, m1, m2, m3)
bt_rr

{'bt_rr': 0.8995671878792302,
 'bt_arr_rr': [0.8950954782552878,
  0.9047682189305981,
  0.9024360993052649,
  0.7820851085032969,
  0.9099977515074462,
  0.8880588659480342,
  0.888529896094769,
  0.9286474567985643,
  0.7443549274754073,
  0.9041411696314656,
  0.8684501951365271,
  0.8953221311969137,
  0.9066492421042313,
  0.8936365934404481,
  0.8897086112149076,
  0.8636844655752582,
  0.8929012936783124,
  0.9073328098746227,
  0.7607746404195312,
  0.8876644876491563,
  0.8970195567337296,
  0.8865806128818529,
  0.8600945902403181,
  0.8903753123280905,
  0.9021971260233119,
  0.8793387276042635,
  0.8905527526190763,
  0.898902067455879,
  0.9116582275508983,
  0.9156473700721312,
  0.917372000533466,
  0.8791637555366583,
  0.9479073334127955,
  0.8802116817135918,
  0.9351401332045786,
  0.8936502044048605,
  0.9509355266273828,
  0.9413917371100284,
  0.8954072967383542,
  0.9105995505469658,
  0.8946849369969933,
  0.8975567386666485,
  0.8765661029104694,
  0.8997857241

In [40]:
# Computing 95% interval for risk ratio while boostrapping error rate matrix
compute_ci_95(bt_rr["bt_arr_rr"])

[0.7708971127593199, 0.9874741092631595]

In [41]:
# Computing 95% interval for risk ratio subgroups while
# bootstrapping error rate matrix
np.apply_along_axis(compute_ci_95, axis=0, arr=bt_rr["sub_arr_rr"])

array([[0.31628614, 0.92217903, 0.06079537, 0.53900034],
       [1.37075198, 0.92217903, 2.39388607, 1.15786023]])

### Combined bootstrapping for RR

In [47]:
def combined_bootstrap_rr(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), do combined bootstrapping to see
    robustness of risk ratio calculations. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterate through 10 of bootstrapped error matrices
    iterations_em = 10
    rr_arr = []
    for iem in range(iterations_em):
        f = open("...", "rb")
        con_matrix = pickle.load(f)
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        # Iterate through 10 bootstrapped (shuffled) dataframes
        iterations_df = 10
        for idf in range(iterations_em):
            bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
            rr, sub_array = risk_ratio_bootstrap(bt_df, model1, model2, model3, res)
            rr_arr.append(rr)
    print("Number of calcs:", len(rr_arr)) # == 100 based on default settings
    print("Mean combined bootstrap rr:", sum(rr_arr) / len(rr_arr))
    return [sum(rr_arr) / len(rr_arr), rr_arr]
        

In [48]:
### Compute 95% invertal for RR while doing combined bootstrapping
combined_rr = combined_bootstrap_rr(merged_df, m1, m2, m3)
compute_ci_95(combined_rr[1])

Number of calcs: 100
Mean combined bootstrap rr: 0.8735481793257273


[0.6752182228746604, 0.9440202895674686]

### Implementing OR

In [49]:
def odds_ratio(dataframe, model1, model2, model3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), calculate the odds ratio as defined by: 
    causal_effect = (P(Y^{a=1}=1) * P(Y^{a=0}=0)) / (P(Y^{a=1}=0) * P(Y^{a=0}=1))
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model5 = P(a | c)
        4) Default prediction is probability of getting 1 due to how statsmodels works
    '''
    
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    # Creating Matrix of Error Adjustments
    confusion = [
                    [8, 0, 2, 1],
                    [4, 4, 3, 0],
                    [1, 0, 14, 1],
                    [1, 0, 1, 61]
                ] # rows represent the ground truth labels and cols represents the predicted labels

    error_mat = [
                    [8/11, 0, 2/11, 1/11],
                    [4/11, 4/11, 3/11, 0],
                    [1/16, 0, 14/16, 1/16],
                    [1/63, 0, 1/63, 61/63]
                ] # rows represent U* and cols represent U
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to either be 1 or 0
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    numerator_a = np.sum(upper_0a * prob_u0_c) + np.sum(upper_1a * prob_u1_c) + \
                  np.sum(upper_2a * prob_u2_c) + np.sum(upper_3a * prob_u3_c)
    numerator_b = np.sum((1 - lower_0b) * prob_u0_c) + np.sum((1 - lower_1b) * prob_u1_c) + \
                  np.sum((1 - lower_2b) * prob_u2_c) + np.sum((1 - lower_3b) * prob_u3_c)
     
    denominator_a = np.sum((1 - upper_0a) * prob_u0_c) + np.sum((1 - upper_1a) * prob_u1_c) + \
                    np.sum((1 - upper_2a) * prob_u2_c) + np.sum((1 - upper_3a) * prob_u3_c)
    denominator_b = np.sum(lower_0b * prob_u0_c) + np.sum(lower_1b * prob_u1_c) + \
                    np.sum(lower_2b * prob_u2_c) + np.sum(lower_3b * prob_u3_c)
    
    numerator = numerator_a * numerator_b
    denominator = denominator_a * denominator_b
    
    return numerator / denominator
    
    

In [43]:
m1, m2, m3 = generate_models(merged_df)
odds_ratio(merged_df, m1, m2, m3)

0.8864763609817276

### Bootstrapping merged dataframe for OR

In [69]:
def bootstrap_merged_data_or(dataframe, m1, m2, m3):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe as well as three trained models 
    from generate_models(), perform bootstrapping by shuffling the merged dataframe for
    odds ratio calculations. Iterations set to 100.
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    iterations = 100
    output = []
    
    for _ in range(iterations):
        bt_data = dataframe.sample(frac=1, replace=True, ignore_index=True)
        or_val = odds_ratio(bt_data, m1, m2, m3)
        output.append(or_val)

    return [sum(output) / len(output), output]

bt_merged_or = bootstrap_merged_data_or(merged_df, m1, m2, m3)
bt_merged_or[0]

0.8851573696534452

In [70]:
# Compute 95% CI for OR while bootstrapping merged dataframe
compute_ci_95(bt_merged_or[1])

[0.8665715709298584, 0.8999256163715614]

### Bootstrapping Error Matrix for OR

In [61]:
def odds_ratio_bootstrap(dataframe, model1, model2, model3, error_mat):
    '''
    Given a pre-procesesed MIMIC + smoking proxy prediction dataframe, three trained models 
    from generate_models(), and an error rate matrix, perform bootstrapping on error-rate matrix for
    odds ratio calculations. Helper function for bootstrap_or()
    The assumptions of this function are:
        1) Smoking proxy predictions are categorical
        2) Treatment variable values are binary --> either 1 for receiving treatment or 0 
           for not receiving treatment
        3) Order for model inputs matter:
            a) model1 = P(y | u*, a, c) --> y ~ u* + a + c
            b) model2 = P(u* | a, c)
            c) model3 = P(a | c)
    '''
    tmp_df = None
    unique_smoking = [1,2,3,4]
    unique_echo = [1,0]
    exp_array = []
    
    inverse = np.linalg.pinv(error_mat)
    
    # Getting P(A, c, y=1, u*) 
    prob_a1_c_y1_u = []
    prob_a0_c_y1_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to be a cateogrical value in [1,2,3,4]
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y1_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y1_u.append(output)
    
    # Getting P(A, c, y=0, u*)
    prob_a1_c_y0_u = []
    prob_a0_c_y0_u = []
    for s in unique_smoking:
        tmp_df = copy.deepcopy(dataframe)
    
        # Presetting the smoking status in the dataframe to either be 1 or 0
        tmp_df["SMOKING_STATUS"] = [s] * tmp_df.shape[0]
        
        for e in unique_echo:
            tmp_tmp_df = copy.deepcopy(tmp_df)
            tmp_tmp_df["echo"] = [e] * tmp_df.shape[0]
            
            prob_1 = 1 - model1.predict(tmp_tmp_df)
            prob_2 = model2.predict(tmp_tmp_df)[:][s-1]
            prob_3 = model3.predict(tmp_tmp_df)
            
            
            if e == 0:
                output = prob_1 * prob_2 * (1 - prob_3)
                prob_a0_c_y0_u.append(output)
            else:
                output = prob_1 * prob_2 * prob_3
                prob_a1_c_y0_u.append(output)
    
    # Getting P(Y=1 | A=1, C, U=0)
    num_0a = prob_a1_c_y1_u[0] * inverse[0][0] + prob_a1_c_y1_u[1] * inverse[1][0] + prob_a1_c_y1_u[2] * \
             inverse[2][0] + prob_a1_c_y1_u[3] * inverse[3][0]
    tmp_0a = prob_a1_c_y0_u[0] * inverse[0][0] + prob_a1_c_y0_u[1] * inverse[1][0] + prob_a1_c_y0_u[2] * \
             inverse[2][0] + prob_a1_c_y0_u[3] * inverse[3][0]
    denom_0a = num_0a + tmp_0a
    upper_0a = num_0a / denom_0a
    
    # Getting P(Y=1 | A=0, C, U=0)
    num_0b = prob_a0_c_y1_u[0] * inverse[0][0] + prob_a0_c_y1_u[1] * inverse[1][0] + prob_a0_c_y1_u[2] * \
             inverse[2][0] + prob_a0_c_y1_u[3] * inverse[3][0]
    tmp_0b = prob_a0_c_y0_u[0] * inverse[0][0] + prob_a0_c_y0_u[1] * inverse[1][0] + prob_a0_c_y0_u[2] * \
             inverse[2][0] + prob_a0_c_y0_u[3] * inverse[3][0]
    denom_0b = num_0b + tmp_0b
    lower_0b = num_0b / denom_0b
    
    
    # Getting P(Y=1 | A=1, C, U=1)
    num_1a = prob_a1_c_y1_u[0] * inverse[0][1] + prob_a1_c_y1_u[1] * inverse[1][1] + prob_a1_c_y1_u[2] * \
             inverse[2][1] + prob_a1_c_y1_u[3] * inverse[3][1]
    tmp_1a = prob_a1_c_y0_u[0] * inverse[0][1] + prob_a1_c_y0_u[1] * inverse[1][1] + prob_a1_c_y0_u[2] * \
             inverse[2][1] + prob_a1_c_y0_u[3] * inverse[3][1]
    denom_1a = num_1a + tmp_1a
    upper_1a = num_1a / denom_1a
    
    # Getting P(Y=1 | A=0, C, U=1)
    num_1b = prob_a0_c_y1_u[0] * inverse[0][1] + prob_a0_c_y1_u[1] * inverse[1][1] + prob_a0_c_y1_u[2] * \
             inverse[2][1] + prob_a0_c_y1_u[3] * inverse[3][1]
    tmp_1b = prob_a0_c_y0_u[0] * inverse[0][1] + prob_a0_c_y0_u[1] * inverse[1][1] + prob_a0_c_y0_u[2] * \
             inverse[2][1] + prob_a0_c_y0_u[3] * inverse[3][1]
    denom_1b = num_1b + tmp_1b
    lower_1b = num_1b / denom_1b
    
    
    # Getting P(Y=1 | A=1, C, U=2)
    num_2a = prob_a1_c_y1_u[0] * inverse[0][2] + prob_a1_c_y1_u[1] * inverse[1][2] + prob_a1_c_y1_u[2] * \
             inverse[2][2] + prob_a1_c_y1_u[3] * inverse[3][2]
    tmp_2a = prob_a1_c_y0_u[0] * inverse[0][2] + prob_a1_c_y0_u[1] * inverse[1][2] + prob_a1_c_y0_u[2] * \
             inverse[2][2] + prob_a1_c_y0_u[3] * inverse[3][2]
    denom_2a = num_2a + tmp_2a
    upper_2a = num_2a / denom_2a
    
    # Getting P(Y=1 | A=0, C, U=2)
    num_2b = prob_a0_c_y1_u[0] * inverse[0][2] + prob_a0_c_y1_u[1] * inverse[1][2] + prob_a0_c_y1_u[2] * \
             inverse[2][2] + prob_a0_c_y1_u[3] * inverse[3][2]
    tmp_2b = prob_a0_c_y0_u[0] * inverse[0][2] + prob_a0_c_y0_u[1] * inverse[1][2] + prob_a0_c_y0_u[2] * \
             inverse[2][2] + prob_a0_c_y0_u[3] * inverse[3][2]
    denom_2b = num_2b + tmp_2b
    lower_2b = num_2b / denom_2b
    
    # Getting P(Y=1 | A=1, C, U=3)
    num_3a = prob_a1_c_y1_u[0] * inverse[0][3] + prob_a1_c_y1_u[1] * inverse[1][3] + prob_a1_c_y1_u[2] * \
             inverse[2][3] + prob_a1_c_y1_u[3] * inverse[3][3]
    tmp_3a = prob_a1_c_y0_u[0] * inverse[0][3] + prob_a1_c_y0_u[1] * inverse[1][3] + prob_a1_c_y0_u[2] * \
             inverse[2][3] + prob_a1_c_y0_u[3] * inverse[3][3]
    denom_3a = num_3a + tmp_3a
    upper_3a = num_3a / denom_3a
    
    # Getting P(Y=1 | A=0, C, U=3)
    num_3b = prob_a0_c_y1_u[0] * inverse[0][3] + prob_a0_c_y1_u[1] * inverse[1][3] + prob_a0_c_y1_u[2] * \
             inverse[2][3] + prob_a0_c_y1_u[3] * inverse[3][3]
    tmp_3b = prob_a0_c_y0_u[0] * inverse[0][3] + prob_a0_c_y0_u[1] * inverse[1][3] + prob_a0_c_y0_u[2] * \
             inverse[2][3] + prob_a0_c_y0_u[3] * inverse[3][3]
    denom_3b = num_3b + tmp_3b
    lower_3b = num_3b / denom_3b
    
    # Getting P(u | c) 
    prob_u0_c = num_0a + tmp_0a + num_0b + tmp_0b
    prob_u1_c = num_1a + tmp_1a + num_1b + tmp_1b
    prob_u2_c = num_2a + tmp_2a + num_2b + tmp_2b
    prob_u3_c = num_3a + tmp_3a + num_3b + tmp_3b
    
    numerator_a = np.sum(upper_0a * prob_u0_c) + np.sum(upper_1a * prob_u1_c) + \
                  np.sum(upper_2a * prob_u2_c) + np.sum(upper_3a * prob_u3_c)
    numerator_b = np.sum((1 - lower_0b) * prob_u0_c) + np.sum((1 - lower_1b) * prob_u1_c) + \
                  np.sum((1 - lower_2b) * prob_u2_c) + np.sum((1 - lower_3b) * prob_u3_c)
     
    denominator_a = np.sum((1 - upper_0a) * prob_u0_c) + np.sum((1 - upper_1a) * prob_u1_c) + \
                    np.sum((1 - upper_2a) * prob_u2_c) + np.sum((1 - upper_3a) * prob_u3_c)
    denominator_b = np.sum(lower_0b * prob_u0_c) + np.sum(lower_1b * prob_u1_c) + \
                    np.sum(lower_2b * prob_u2_c) + np.sum(lower_3b * prob_u3_c)
    
    numerator = numerator_a * numerator_b
    denominator = denominator_a * denominator_b
    
    return numerator / denominator

In [62]:
def bootstrap_or(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 5 models generated from generate_models(), bootstrap the testing set 
    for n2c2 2006 smoking dataset to get different error rate matrices to test robustness of 
    the odds ratio casual effect.
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterating through the bootstrapped confusion matrices 
    # "iterations" var depends on how many bootstrapped confusion matrics were generated
    # Default in predict_bootstrap_2006.py is 10
    iterations = 100 
    o_r_arr = []
    for x in range(iterations):
        # Access each pickle file containing the confusion matrix
        f = open("...", "rb")  # First input should be the bootstrapped matrices (pkl file)
        con_matrix = pickle.load(f)
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        o_r = odds_ratio_bootstrap(dataframe, model1, model2, model3, res)
        o_r_arr.append(o_r)
    
    return [sum(o_r_arr) / len(o_r_arr), o_r_arr]

In [63]:
m1, m2, m3 = generate_models(merged_df)
bt_or = bootstrap_or(merged_df, m1, m2, m3)
bt_or[0]

0.9196165710847475

In [64]:
# Computing 95% interval for odds ratio while boostrapping error rate matrix
compute_ci_95(bt_or[1])

[0.7734094843134075, 0.9803855037359114]

### Combined Bootstrapping for OR

In [67]:
def combined_bootstrap_or(dataframe, model1, model2, model3):
    '''
    Given a dataframe and 3 models generated from generate_models(), do combined bootstrapping to see
    robustness of odds ratio calculations. Iterations set to 100
    Utilize predict_bootstrap_2006.py to generate pickle files that store the confusion matrices.
    '''
    
    # Iterate through 10 of bootstrapped error matrices
    iterations_em = 10
    or_arr = []
    for iem in range(iterations_em):
        f = open("...", "rb")
        con_matrix = pickle.load(f)
        res = con_matrix/con_matrix.sum(axis=1)[:,None]
        
        # Iterate through 10 bootstrapped (shuffled) dataframes
        iterations_df = 10
        for idf in range(iterations_df):
            bt_df = dataframe.sample(frac=1, replace=True, ignore_index=True)
            or_v = odds_ratio_bootstrap(bt_df, model1, model2, model3, res)
            or_arr.append(or_v)
    print(len(or_arr)) # == 100 based on default settings
    print("Mean combined bootstrap or_v:", sum(or_arr) / len(or_arr))
    return [sum(or_arr) / len(or_arr), or_arr]

In [68]:
### Compute 95% invertal for RR while doing combined bootstrapping
combined_or = combined_bootstrap_or(merged_df, m1, m2, m3)
compute_ci_95(combined_or[1])

100
Mean combined bootstrap or_v: 0.883528044221483


[0.5818542201286497, 0.955810700152887]